# Dependencies

In [1]:
#ETL and Webscraping Dependencies 
import pandas as pd
import requests
import json
import os
import time

from datetime import datetime as dt
from bs4 import BeautifulSoup

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

In [2]:
USERNAME = "postgres"
PASSWORD = "georgia1"
rds_connection_string = f"{USERNAME}:{PASSWORD}@localhost:5432/EColi"
engine = create_engine(f'postgresql://{rds_connection_string}')

# Store CSV Into Dataframe

In [3]:
file = "Food_Inspections.csv"
restaurant_df = pd.read_csv(file)
restaurant_df.head()

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location
0,2079132,MARRIOT MARQUIS CHICAGO,MARRIOT MARQUIS CHICAGO,2517328.0,Restaurant,Risk 1 (High),2121 S PRAIRIE AVE,CHICAGO,IL,60616.0,8/28/17,License,Pass,NaN,41.853651,-87.620534,"(41.853650885040594, -87.62053358114167)"
1,2079129,JET'S PIZZA,JET'S PIZZA,2522268.0,Restaurant,Risk 2 (Medium),1025 W MADISON ST,CHICAGO,IL,60607.0,8/28/17,License,Not Ready,NaN,41.881572,-87.653052,"(41.88157249576794, -87.65305233593274)"
2,2079125,ROOM 1520,ROOM 1520,2446638.0,Special Event,Risk 3 (Low),301 N JUSTINE ST,CHICAGO,IL,60607.0,8/28/17,License Re-Inspection,Not Ready,8. SANITIZING RINSE FOR EQUIPMENT AND UTENSILS...,41.886577,-87.665328,"(41.88657652150854, -87.66532812140231)"
3,2079123,MARRIOT MARQUIS CHICAGO,MARRIOT MARQUIS CHICAGO,2517338.0,Restaurant,Risk 1 (High),2121 S PRAIRIE AVE,CHICAGO,IL,60616.0,8/28/17,License,Pass,"35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTR...",41.853651,-87.620534,"(41.853650885040594, -87.62053358114167)"
4,2079105,CHARTWELLS,CICS WEST BELDEN CAMPUS,2549079.0,CHARTER SCHOOL,Risk 1 (High),2245 N MCVICKER AVE,CHICAGO,IL,60639.0,8/28/17,License Re-Inspection,Pass,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...,41.921675,-87.776711,"(41.921675488910864, -87.7767113569357)"


# Cleaning/Munging

In [4]:
#Filter to failed 'Results'
fail_df = restaurant_df[restaurant_df.Results == 'Fail']

In [5]:
fail_df.drop(columns=['Inspection ID', 'AKA Name', 'License #', 'City', 'State', 'Zip', 'Inspection Type', 'Violations', 'Location'])

,DBA Name,Facility Type,Risk,Address,Inspection Date,Results,Latitude,Longitude
5,SHRIMP'S FISH CHICKEN,Restaurant,Risk 2 (Medium),4355 S COTTAGE GROVE AVE,8/28/17,Fail,41.815197,-87.606531
6,ART'S DRIVE INN,Restaurant,Risk 1 (High),1333 W NORTH AVE,8/25/17,Fail,41.910601,-87.661863
7,VINCENT,Restaurant,Risk 1 (High),1475 W BALMORAL AVE,8/25/17,Fail,41.979744,-87.667738
12,PHO 777 RESTAURANT,Restaurant,Risk 1 (High),1063-1065 W ARGYLE ST,8/25/17,Fail,41.973167,-87.657438
15,FULL SHILLING,Restaurant,Risk 1 (High),3724-3726 N CLARK ST,8/25/17,Fail,41.949706,-87.658701
...,...,...,...,...,...,...,...,...
153792,CITGO,Grocery Store,Risk 3 (Low),11500 S Halsted ST,1/5/10,Fail,41.685026,-87.642297
153798,"MONTICELLO FOOD MART, INC",Grocery Store,Risk 2 (Medium),1000 N MONTICELLO AVE,1/5/10,Fail,41.899186,-87.717804
153800,SUPER DOG'S SUBMARINES INC,Restaurant,Risk 2 (Medium),1442 W 103RD ST,1/5/10,Fail,41.706852,-87.658946
153803,WHIPPLE STORE,Grocery Store,Risk 3 (Low),4301 N WHIPPLE ST,1/5/10,Fail,41.959489,-87.704380


In [7]:
fail_df = fail_df.rename(columns={'DBA Name': 'name', 'Facility Type': 'facility', 'Risk': 'risk', 'Address': 'address', 'Inspection Date': 'date', 'Results': 'results', 'Latitude':'latitude', 'Longitude': 'longitude'})

In [8]:
fail_df['id'] = fail_df.index

In [10]:
fail_df = fail_df[['id', 'name', 'facility', 'risk', 'address', 'date', 'results', 'latitude','longitude']]

In [11]:
#Store csv's into a dataframe
file = "beach-e.-coli-predictions.csv"
beach_df = pd.read_csv(file)
beach_df.head()

,Beach Name,Date,Prediction Source,Predicted Level,RecordID,Latitude,Longitude,Location
0,12th Street,2017-05-26T00:00:00,DNA Model,160.0,12thStreet20170526,41.8638,-87.6082,"{'needs_recoding': False, 'longitude': '-87.60..."
1,Margaret T Burroughs (31st),2017-05-26T00:00:00,DNA Model,184.6,MargaretTBurroughs31st20170526,41.8393,-87.6072,"{'needs_recoding': False, 'longitude': '-87.60..."
2,Oakwood,2017-05-26T00:00:00,DNA Model,163.7,Oakwood20170526,41.8211,-87.5976,"{'needs_recoding': False, 'longitude': '-87.59..."
3,57th Street,2017-05-26T00:00:00,DNA Model,82.7,57thStreet20170526,41.7911,-87.5797,"{'needs_recoding': False, 'longitude': '-87.57..."
4,Hartigan (Albion),2017-05-26T00:00:00,DNA Model,79.5,HartiganAlbion20170526,42.0027,-87.6564,"{'needs_recoding': False, 'longitude': '-87.65..."


In [12]:
beach_df.dropna(subset=['Date'])

,Beach Name,Date,Prediction Source,Predicted Level,RecordID,Latitude,Longitude,Location
0,12th Street,2017-05-26T00:00:00,DNA Model,160.0,12thStreet20170526,41.8638,-87.6082,"{'needs_recoding': False, 'longitude': '-87.60..."
1,Margaret T Burroughs (31st),2017-05-26T00:00:00,DNA Model,184.6,MargaretTBurroughs31st20170526,41.8393,-87.6072,"{'needs_recoding': False, 'longitude': '-87.60..."
2,Oakwood,2017-05-26T00:00:00,DNA Model,163.7,Oakwood20170526,41.8211,-87.5976,"{'needs_recoding': False, 'longitude': '-87.59..."
3,57th Street,2017-05-26T00:00:00,DNA Model,82.7,57thStreet20170526,41.7911,-87.5797,"{'needs_recoding': False, 'longitude': '-87.57..."
4,Hartigan (Albion),2017-05-26T00:00:00,DNA Model,79.5,HartiganAlbion20170526,42.0027,-87.6564,"{'needs_recoding': False, 'longitude': '-87.65..."
...,...,...,...,...,...,...,...,...
2366,Oak Street,2018-08-06T00:00:00,DNA Model,124.5,OakStreet20180806,41.9032,-87.6235,"{'needs_recoding': False, 'longitude': '-87.62..."
2367,Ohio Street,2018-08-06T00:00:00,DNA Model,223.5,OhioStreet20180806,41.8935,-87.6152,"{'needs_recoding': False, 'longitude': '-87.61..."
2368,Osterman,2018-08-06T00:00:00,DNA Model,288.6,Osterman20180806,41.9877,-87.6545,"{'needs_recoding': False, 'longitude': '-87.65..."
2369,Rogers,2018-08-06T00:00:00,DNA Model,136.0,Rogers20180806,42.0213,-87.6666,"{'needs_recoding': False, 'longitude': '-87.66..."


In [13]:
#Create level bins
bins = [0, 99, 199, 299, 1000]

#Create the names for the level bins
group_names = ["Safe", "Warning Zone", "Bad", "Danger"]

#Add groups
beach_df["ecoli_level"]=pd.cut(beach_df["Predicted Level"], bins, labels=group_names)

In [14]:
#Drop 'Location', 'RecordID', and 'Prediction Source'
beach_df = beach_df.drop(columns=['Location', 'RecordID', 'Prediction Source'])

In [15]:
beach_df

,Beach Name,Date,Predicted Level,Latitude,Longitude,ecoli_level
0,12th Street,2017-05-26T00:00:00,160.0,41.8638,-87.6082,Warning Zone
1,Margaret T Burroughs (31st),2017-05-26T00:00:00,184.6,41.8393,-87.6072,Warning Zone
2,Oakwood,2017-05-26T00:00:00,163.7,41.8211,-87.5976,Warning Zone
3,57th Street,2017-05-26T00:00:00,82.7,41.7911,-87.5797,Safe
4,Hartigan (Albion),2017-05-26T00:00:00,79.5,42.0027,-87.6564,Safe
...,...,...,...,...,...,...
2366,Oak Street,2018-08-06T00:00:00,124.5,41.9032,-87.6235,Warning Zone
2367,Ohio Street,2018-08-06T00:00:00,223.5,41.8935,-87.6152,Bad
2368,Osterman,2018-08-06T00:00:00,288.6,41.9877,-87.6545,Bad
2369,Rogers,2018-08-06T00:00:00,136.0,42.0213,-87.6666,Warning Zone


In [16]:
#Change 'Date' to digestable format
beach_df['Date'].replace(regex=True,inplace=True,to_replace=r'T00:00:00',value=r'')
beach_df['Date'] = pd.to_datetime(beach_df['Date']).apply(lambda x:x.strftime('%m/%d/%Y'))

In [17]:
beach_df.head()

,Beach Name,Date,Predicted Level,Latitude,Longitude,ecoli_level
0,12th Street,05/26/2017,160.0,41.8638,-87.6082,Warning Zone
1,Margaret T Burroughs (31st),05/26/2017,184.6,41.8393,-87.6072,Warning Zone
2,Oakwood,05/26/2017,163.7,41.8211,-87.5976,Warning Zone
3,57th Street,05/26/2017,82.7,41.7911,-87.5797,Safe
4,Hartigan (Albion),05/26/2017,79.5,42.0027,-87.6564,Safe


In [18]:
beach_df.rename(columns={'Beach Name':'beach_name', 'Date':'date', 'Predicted Level':'predicted_level', 'Latitude':'latitude', 'Longitude':'longitude', 'ecoli_level':'ecoli'}, inplace=True)

In [19]:
beach_df.head()

,beach_name,date,predicted_level,latitude,longitude,ecoli
0,12th Street,05/26/2017,160.0,41.8638,-87.6082,Warning Zone
1,Margaret T Burroughs (31st),05/26/2017,184.6,41.8393,-87.6072,Warning Zone
2,Oakwood,05/26/2017,163.7,41.8211,-87.5976,Warning Zone
3,57th Street,05/26/2017,82.7,41.7911,-87.5797,Safe
4,Hartigan (Albion),05/26/2017,79.5,42.0027,-87.6564,Safe


In [20]:
beach_df['id'] = beach_df.index

In [21]:
beach_df = beach_df[['id', 'beach_name', 'date', 'predicted_level', 'latitude', 'longitude', 'ecoli']]

In [26]:
beach_df.head()

,id,beach_name,date,predicted_level,latitude,longitude,ecoli
0,0,12th Street,05/26/2017,160.0,41.8638,-87.6082,Warning Zone
1,1,Margaret T Burroughs (31st),05/26/2017,184.6,41.8393,-87.6072,Warning Zone
2,2,Oakwood,05/26/2017,163.7,41.8211,-87.5976,Warning Zone
3,3,57th Street,05/26/2017,82.7,41.7911,-87.5797,Safe
4,4,Hartigan (Albion),05/26/2017,79.5,42.0027,-87.6564,Safe


In [27]:
fail_df.head()

,id,name,facility,risk,address,date,results,latitude,longitude
5,5,SHRIMP'S FISH CHICKEN,Restaurant,Risk 2 (Medium),4355 S COTTAGE GROVE AVE,8/28/17,Fail,41.815197,-87.606531
6,6,ART'S DRIVE INN,Restaurant,Risk 1 (High),1333 W NORTH AVE,8/25/17,Fail,41.910601,-87.661863
7,7,VINCENT,Restaurant,Risk 1 (High),1475 W BALMORAL AVE,8/25/17,Fail,41.979744,-87.667738
12,12,PHO 777 RESTAURANT,Restaurant,Risk 1 (High),1063-1065 W ARGYLE ST,8/25/17,Fail,41.973167,-87.657438
15,15,FULL SHILLING,Restaurant,Risk 1 (High),3724-3726 N CLARK ST,8/25/17,Fail,41.949706,-87.658701


In [22]:
# Confirm tables
engine.table_names()

['beach_ecoli', 'inspections']

In [28]:
beach_df.to_sql(name='beach_ecoli', con=engine, if_exists='append', index=False)

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "beach_ecoli_pkey"
DETAIL:  Key (id)=(0) already exists.

[SQL: INSERT INTO beach_ecoli (id, beach_name, date, predicted_level, latitude, longitude, ecoli) VALUES (%(id)s, %(beach_name)s, %(date)s, %(predicted_level)s, %(latitude)s, %(longitude)s, %(ecoli)s)]
[parameters: ({'id': 0, 'beach_name': '12th Street', 'date': '05/26/2017', 'predicted_level': 160.0, 'latitude': 41.8638, 'longitude': -87.6082, 'ecoli': 'Warning Zone'}, {'id': 1, 'beach_name': 'Margaret T Burroughs (31st)', 'date': '05/26/2017', 'predicted_level': 184.6, 'latitude': 41.8393, 'longitude': -87.6072, 'ecoli': 'Warning Zone'}, {'id': 2, 'beach_name': 'Oakwood', 'date': '05/26/2017', 'predicted_level': 163.7, 'latitude': 41.8211, 'longitude': -87.5976, 'ecoli': 'Warning Zone'}, {'id': 3, 'beach_name': '57th Street', 'date': '05/26/2017', 'predicted_level': 82.7, 'latitude': 41.7911, 'longitude': -87.5797, 'ecoli': 'Safe'}

In [29]:
fail_df.to_sql(name='inspections', con=engine, if_exists='append', index=False)

In [30]:
#check to make sure data is in SQL
pd.read_sql_query('select * from beach_ecoli', con=engine).head()

,id,beach_name,date,predicted_level,latitude,longitude,ecoli
0,0,12th Street,2017-05-26,160.0,41.8638,-87.6082,Warning Zone
1,1,Margaret T Burroughs (31st),2017-05-26,184.6,41.8393,-87.6072,Warning Zone
2,2,Oakwood,2017-05-26,163.7,41.8211,-87.5976,Warning Zone
3,3,57th Street,2017-05-26,82.7,41.7911,-87.5797,Safe
4,4,Hartigan (Albion),2017-05-26,79.5,42.0027,-87.6564,Safe


In [31]:
#check to make sure data is in SQL
pd.read_sql_query('select * from inspections', con=engine).head()

,id,name,facility,risk,address,date,results,latitude,longitude
0,5,SHRIMP'S FISH CHICKEN,Restaurant,Risk 2 (Medium),4355 S COTTAGE GROVE AVE,2017-08-28,Fail,41.815197,-87.606531
1,6,ART'S DRIVE INN,Restaurant,Risk 1 (High),1333 W NORTH AVE,2017-08-25,Fail,41.910601,-87.661863
2,7,VINCENT,Restaurant,Risk 1 (High),1475 W BALMORAL AVE,2017-08-25,Fail,41.979744,-87.667738
3,12,PHO 777 RESTAURANT,Restaurant,Risk 1 (High),1063-1065 W ARGYLE ST,2017-08-25,Fail,41.973167,-87.657438
4,15,FULL SHILLING,Restaurant,Risk 1 (High),3724-3726 N CLARK ST,2017-08-25,Fail,41.949706,-87.658701


In [43]:
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()

['beach_ecoli', 'inspections']

In [44]:
Ecoli = Base.classes.beach_ecoli

In [45]:
Inspections = Base.classes.inspections

In [46]:
session = Session(engine)

In [51]:
Severe = session.query(Ecoli).filter(Ecoli.ecoli == 'Warning Zone').all()
for ecoli in Severe:
    print(f"Ecoli: {ecoli.ecoli}, Date: {ecoli.date}")

Ecoli: Warning Zone, Date: 2017-05-26
Ecoli: Warning Zone, Date: 2017-05-26
Ecoli: Warning Zone, Date: 2017-05-26
Ecoli: Warning Zone, Date: 2017-05-26
Ecoli: Warning Zone, Date: 2017-05-26
Ecoli: Warning Zone, Date: 2017-05-29
Ecoli: Warning Zone, Date: 2017-05-29
Ecoli: Warning Zone, Date: 2017-05-31
Ecoli: Warning Zone, Date: 2017-05-31
Ecoli: Warning Zone, Date: 2017-06-01
Ecoli: Warning Zone, Date: 2017-06-01
Ecoli: Warning Zone, Date: 2017-06-01
Ecoli: Warning Zone, Date: 2017-06-01
Ecoli: Warning Zone, Date: 2017-06-02
Ecoli: Warning Zone, Date: 2017-06-02
Ecoli: Warning Zone, Date: 2017-06-02
Ecoli: Warning Zone, Date: 2017-06-02
Ecoli: Warning Zone, Date: 2017-06-03
Ecoli: Warning Zone, Date: 2017-06-03
Ecoli: Warning Zone, Date: 2017-06-03
Ecoli: Warning Zone, Date: 2017-06-03
Ecoli: Warning Zone, Date: 2017-06-03
Ecoli: Warning Zone, Date: 2017-06-03
Ecoli: Warning Zone, Date: 2017-06-03
Ecoli: Warning Zone, Date: 2017-06-03
Ecoli: Warning Zone, Date: 2017-06-05
Ecoli: Warni

In [52]:
 inspector = inspect(engine)
inspector.get_table_names()

['beach_ecoli', 'inspections']

In [54]:
columns = inspector.get_columns('beach_ecoli')
for c in columns:
    print(c['name'], c["type"])

id INTEGER
beach_name TEXT
date DATE
predicted_level DOUBLE PRECISION
latitude DOUBLE PRECISION
longitude DOUBLE PRECISION
ecoli TEXT


In [58]:
sel = [Ecoli.ecoli, Inspections.name, Inspections.risk, Inspections.results]
same_date = session.query(*sel).filter(Ecoli.date == Inspections.date).limit(100).all()

for record in same_date:
    (beach_ecoli_ecoli, inspections_name, inspections_risk, inspections_results) = record
    print(
        f"The ecoli level was'{beach_ecoli_ecoli}'"
        f"when '{inspections_name}' had the level '{inspections_risk}' and got the result '{inspections_results}'.")

The ecoli level was'Safe'when 'SHRIMP'S FISH CHICKEN' had the level 'Risk 2 (Medium)' and got the result 'Fail'.
The ecoli level was'Safe'when 'SHRIMP'S FISH CHICKEN' had the level 'Risk 2 (Medium)' and got the result 'Fail'.
The ecoli level was'Safe'when 'SHRIMP'S FISH CHICKEN' had the level 'Risk 2 (Medium)' and got the result 'Fail'.
The ecoli level was'Safe'when 'SHRIMP'S FISH CHICKEN' had the level 'Risk 2 (Medium)' and got the result 'Fail'.
The ecoli level was'Safe'when 'SHRIMP'S FISH CHICKEN' had the level 'Risk 2 (Medium)' and got the result 'Fail'.
The ecoli level was'Safe'when 'SHRIMP'S FISH CHICKEN' had the level 'Risk 2 (Medium)' and got the result 'Fail'.
The ecoli level was'Safe'when 'SHRIMP'S FISH CHICKEN' had the level 'Risk 2 (Medium)' and got the result 'Fail'.
The ecoli level was'Safe'when 'SHRIMP'S FISH CHICKEN' had the level 'Risk 2 (Medium)' and got the result 'Fail'.
The ecoli level was'Safe'when 'SHRIMP'S FISH CHICKEN' had the level 'Risk 2 (Medium)' and got th